In [14]:
import json

from hybrik.utils.bbox import bbox_clip_xyxy, bbox_xywh_to_xyxy
from hybrik.utils.pose_utils import (cam2pixel_matrix, pixel2cam_matrix,
                                     reconstruction_error)
from hybrik.utils.presets import (SimpleTransform3DSMPL,
                                  SimpleTransform3DSMPLCam)

import numpy as np

In [1]:
items = []
labels = []
ann_file = '/home/aiteam/tykim/nia-3d/HybrIK/data/3dhp/annotation_mpi_inf_3dhp_train_v2.json'

In [3]:
with open(ann_file, 'r') as fid:
    database = json.load(fid)

In [6]:
database['images'][0].keys()

dict_keys(['id', 'file_name', 'width', 'height', 'subject', 'seq_idx', 'cam_idx', 'frame_idx', 'cam_param'])

In [7]:
database['annotations'][0].keys()

dict_keys(['id', 'image_id', 'category_id', 'iscrowd', 'keypoints_world', 'keypoints_cam', 'keypoints_img', 'keypoints_vis', 'bbox', 'area'])

In [9]:
len(database['annotations'][0]['keypoints_cam'])

28

In [11]:
joints_name = ('spine3', 'spine4', 'spine2', 'spine', 'pelvis',                         # 4
                   'neck', 'head', 'head_top',                                              # 7
                   'left_clavicle', 'left_shoulder', 'left_elbow',                          # 10
                   'left_wrist', 'left_hand', 'right_clavicle',                             # 13
                   'right_shoulder', 'right_elbow', 'right_wrist',                          # 16
                   'right_hand', 'left_hip', 'left_knee',                                   # 19
                   'left_ankle', 'left_foot', 'left_toe',                                   # 22
                   'right_hip', 'right_knee', 'right_ankle', 'right_foot', 'right_toe')     # 27


In [12]:
root_idx = joints_name.index('pelvis')
num_joints = 28

In [16]:

for ann_image, ann_annotations in zip(database['images'], database['annotations']):
    ann = dict()
    for k, v in ann_image.items():
        assert k not in ann.keys()
        ann[k] = v
    for k, v in ann_annotations.items():
        ann[k] = v

    image_id = ann['image_id']

    width, height = ann['width'], ann['height']
    xmin, ymin, xmax, ymax = bbox_clip_xyxy(
        bbox_xywh_to_xyxy(ann['bbox']), width, height)

    intrinsic_param = np.array(ann['cam_param']['intrinsic_param'], dtype=np.float32)

    f = np.array([intrinsic_param[0, 0], intrinsic_param[1, 1]], dtype=np.float32)
    c = np.array([intrinsic_param[0, 2], intrinsic_param[1, 2]], dtype=np.float32)

    joint_cam = np.array(ann['keypoints_cam'])

    joint_img = cam2pixel_matrix(joint_cam, intrinsic_param)
    joint_img[:, 2] = joint_img[:, 2] - joint_cam[root_idx, 2]
    joint_vis = np.ones((num_joints, 3))

    root_cam = joint_cam[root_idx]


    labels.append({
        'bbox': (xmin, ymin, xmax, ymax),
        'img_id': image_id,
        'img_name': ann['file_name'],
        'width': width,
        'height': height,
        'joint_img': joint_img,
        'joint_vis': joint_vis,
        'joint_cam': joint_cam,
        'root_cam': root_cam,
        'intrinsic_param': intrinsic_param,
        'f': f,
        'c': c
    })

In [17]:
labels[0]

{'bbox': (958.7267074584961,
  826.9013595581055,
  1074.279151916504,
  1711.6228103637695),
 'img_id': 0,
 'img_name': 'S1/Seq1/images/S1_Seq1_V0/img_S1_Seq1_V0_000001.jpg',
 'width': 2048,
 'height': 2048,
 'joint_img': array([[1014.16733982, 1091.08414691,  -13.65991211],
        [1006.20655777, 1048.60433302,  -21.75      ],
        [1019.90739724, 1122.38102961,   -8.19995117],
        [1019.93871745, 1122.61410108,   -8.16845703],
        [1025.3798342 , 1213.84595922,    0.        ],
        [1023.9380922 , 1019.5250705 ,  -15.7800293 ],
        [1029.56238731,  984.93115078,  -17.        ],
        [1018.94669147,  915.37349387,  -27.85009766],
        [1029.04544694, 1033.82473233,   30.87988281],
        [1016.11168414, 1053.96442412,  110.31005859],
        [1006.99957376, 1054.79064688,  426.11987305],
        [1008.26216335, 1061.62873669,  670.26000977],
        [1005.533607  , 1068.65020787,  756.55981445],
        [1036.08123748, 1034.89639901,  -50.36010742],
        

In [1]:
"""Script for multi-gpu training."""
import os
import pickle as pk
import random
import sys

import numpy as np
import torch
import torch.multiprocessing as mp
import torch.utils.data
from torch.nn.utils import clip_grad_norm_
from torch.nn.utils import clip_grad

from hybrik.datasets import MixDataset, MixDatasetCam, PW3D, MixDataset2Cam, H36NIADataset, H36NIADatasetCam, NIA, Mscoco, H36MCoCoDataset, HP3D, H36mSMPL, NIA3D, NIA3DMixDataset


from hybrik.utils.bbox import bbox_clip_xyxy, bbox_xywh_to_xyxy
from hybrik.utils.pose_utils import cam2pixel, pixel2cam, reconstruction_error

import yaml
from easydict import EasyDict as edict

config_file = '/home/aiteam/tykim/nia-3d/HybrIK/configs/256x192_adam_lr1e-3-res34_smpl_3d_cam_2x_mix_nia.yaml'

with open(config_file) as f:
  cfg = edict(yaml.load(f, Loader=yaml.FullLoader))

/home/aiteam/miniconda3/envs/hybrik3/lib/python3.8/site-packages/tqdm-4.64.1-py3.8.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
db2 = HP3D(
            cfg=cfg,
            root = '/home/aiteam/tykim/nia-3d/HybrIK/data/3dhp', 
            ann_file='train_v2',
            train=True)

In [4]:
img, target, img_id, bbox =  db2[2]

In [6]:
target['target'].shape

torch.Size([84])

In [5]:
target

{'type': '2d_data',
 'target': tensor([ 0.1115, -0.1090, -0.0063,  0.1422, -0.1312, -0.0134,  0.0891, -0.0924,
         -0.0024,  0.0889, -0.0923, -0.0023,  0.0306, -0.0365,  0.0000,  0.1495,
         -0.1604, -0.0105,  0.1671, -0.1860, -0.0118,  0.2158, -0.2247, -0.0153,
          0.1306, -0.1587, -0.0230,  0.1258, -0.1527, -0.0672,  0.1053, -0.1363,
         -0.2082,  0.0823, -0.1412, -0.3173,  0.0726, -0.1384, -0.3570,  0.1400,
         -0.1539,  0.0124,  0.1389, -0.1394,  0.0536,  0.1413, -0.1305,  0.1970,
          0.1363, -0.1262,  0.3080,  0.1358, -0.1182,  0.3465,  0.0182, -0.0443,
         -0.0511, -0.1079,  0.1084, -0.0200, -0.1866,  0.2385,  0.0011, -0.2188,
          0.2218, -0.0019, -0.2239,  0.2135, -0.0024,  0.0422, -0.0291,  0.0511,
         -0.0907,  0.1126,  0.0377, -0.1748,  0.2442,  0.0289, -0.1971,  0.2209,
          0.0480, -0.2009,  0.2123,  0.0518]),
 'target_weight': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.